In [2]:
import time
import pickle
import datetime
import holidays
import numpy as np
import pandas as pd
import seaborn as sns
import pandas_profiling
import matplotlib.pyplot as plt
from math import sqrt
from geopy.distance import geodesic 
from sklearn.metrics import r2_score
from geopy.geocoders import Nominatim
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
def count(df):
  counter = {}
  for i in df:
    if i in counter:
      counter[i] += 1
    else:
      counter[i] = 1
  return counter

In [0]:
def heatMap(df):
    #Create Correlation df
    corr = df.corr()
    #Plot figsize
    fig, ax = plt.subplots(figsize=(10, 10))
    #Generate Color Map
    colormap = sns.diverging_palette(220, 10, as_cmap=True)
    #Generate Heat Map, allow annotations and place floats in map
    sns.heatmap(corr, cmap=colormap, annot=True, fmt=".2f")
    #Apply xticks
    plt.xticks(range(len(corr.columns)), corr.columns);
    #Apply yticks
    plt.yticks(range(len(corr.columns)), corr.columns)
    #show plot
    plt.show()

# Time Pass

In [0]:
df_2012_q1_w = pd.read_csv("/content/drive/My Drive/SEM2/MP/dataset/2012_Q1_withweather.csv")
df_2012_q1_w['Start date'] = pd.to_datetime(df_2012_q1_w['Start date'])
df_2012_q2_w = pd.read_csv("/content/drive/My Drive/SEM2/MP/dataset/2012_Q2_withweather.csv")
df_2012_q2_w['Start date'] = pd.to_datetime(df_2012_q2_w['Start date'])
df_2012_q3_w = pd.read_csv("/content/drive/My Drive/SEM2/MP/dataset/2012_Q3_withweather.csv")
df_2012_q3_w['Start date'] = pd.to_datetime(df_2012_q3_w['Start date'])
df_2012_q4_w = pd.read_csv("/content/drive/My Drive/SEM2/MP/dataset/2012_Q4_withweather.csv")
df_2012_q4_w['Start date'] = pd.to_datetime(df_2012_q4_w['Start date'])

In [0]:
df_2012_d = pd.read_csv("/content/drive/My Drive/SEM2/MP/dataset/2012_with_distance.csv")
df_2012_d['Start date'] = pd.to_datetime(df_2012_d['Start date'])

In [0]:
df_2012_q4_w.head()

,Unnamed: 0,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,weathersit,temp,atemp,hum,windspeed
0,0,611,2012-10-01 00:02:06,2012-10-01 00:12:17,31230,Metro Center / 12th & G St NW,31212,21st & M St NW,W00311,Member,1,0.46,0.4545,0.72,0.1045
1,1,612,2012-10-01 00:02:52,2012-10-01 00:13:05,31624,North Capitol St & F St NW,31251,12th & L St NW,W00137,Member,1,0.46,0.4545,0.72,0.1045
2,2,734,2012-10-01 00:06:34,2012-10-01 00:18:48,31234,20th & O St NW / Dupont South,31223,Convention Center / 7th & M St NW,W20125,Member,1,0.46,0.4545,0.72,0.1045
3,3,219,2012-10-01 00:07:27,2012-10-01 00:11:06,31106,Calvert & Biltmore St NW,31104,Adams Mill & Columbia Rd NW,W01451,Member,1,0.46,0.4545,0.72,0.1045
4,4,924,2012-10-01 00:07:28,2012-10-01 00:22:53,31109,7th & T St NW,31615,6th & H St NE,W00441,Member,1,0.46,0.4545,0.72,0.1045


In [0]:
print(df_2012_q1_w.shape[0]+df_2012_q2_w.shape[0]+df_2012_q3_w.shape[0]+df_2012_q4_w.shape[0])

2028911


In [0]:
df_2012_d.shape[0]

2028911

In [0]:
df_2012_wd = pd.concat([df_2012_q1_w, df_2012_q2_w, df_2012_q3_w, df_2012_q4_w], ignore_index=True)
df_2012_wd['distance'] = df_2012_d['Distance']

In [0]:
df_2012_wd.head()

,Unnamed: 0,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,weathersit,temp,atemp,hum,windspeed,distance
0,0,475,2012-01-01 00:04:00,2012-01-01 00:11:56,31245,7th & R St NW / Shaw Library,31109,7th & T St NW,W01412,Member,1,0.36,0.3788,0.66,0.0,0.192336
1,1,1162,2012-01-01 00:10:05,2012-01-01 00:29:28,31400,Georgia & New Hampshire Ave NW,31103,16th & Harvard St NW,W00524,Casual,1,0.36,0.3788,0.66,0.0,0.941258
2,2,1145,2012-01-01 00:10:23,2012-01-01 00:29:28,31400,Georgia & New Hampshire Ave NW,31103,16th & Harvard St NW,W00235,Member,1,0.36,0.3788,0.66,0.0,0.941258
3,3,485,2012-01-01 00:15:41,2012-01-01 00:23:46,31101,14th & V St NW,31602,Park Rd & Holmead Pl NW,W00864,Member,1,0.36,0.3788,0.66,0.0,0.912716
4,4,471,2012-01-01 00:15:42,2012-01-01 00:23:34,31102,11th & Kenyon St NW,31109,7th & T St NW,W00995,Member,1,0.36,0.3788,0.66,0.0,1.009765


In [0]:
df_2012_wd.to_csv("2012_wd.csv")


# Predicting Hourly Count

In [0]:
df_201112_w = pd.read_csv("/content/drive/My Drive/SEM2/MP/dataset/hour.csv")
# df_2012_wd['Start date'] = pd.to_datetime(df_2012_wd['Start date'])
df_201112_w.head(3)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32


In [0]:
df_2012_w = df_201112_w.loc[(df_201112_w['dteday']>='2012-01-01')&(df_201112_w['dteday']<'2013-01-01')]
df_2012_w.head(3)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
8645,8646,2012-01-01,1,1,1,0,0,0,0,1,0.36,0.3788,0.66,0.0000,5,43,48
8646,8647,2012-01-01,1,1,1,1,0,0,0,1,0.36,0.3485,0.66,0.1343,15,78,93
8647,8648,2012-01-01,1,1,1,2,0,0,0,1,0.32,0.3485,0.76,0.0000,16,59,75


In [0]:
X = df_2012_w[['yr','hr','mnth','season','holiday','weekday','workingday',"temp","hum",'windspeed','weathersit']]
Y = df_2012_w['cnt']

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state=42)

In [0]:
lin_model = LinearRegression(n_jobs=-1)
lin_model.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [0]:
y_test_predict = lin_model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
r2 = r2_score(Y_test, y_test_predict)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

The model performance for testing set
--------------------------------------
RMSE is 170.00864940893265
R2 score is 0.3522249515362025


# Predicting Duration

In [0]:
df_2012_predDuration = pd.read_csv('/content/drive/My Drive/SEM2/MP/dataset/2012_predDuration.csv')
df_2012_predDuration.head(1)

,Unnamed: 0,Duration,Start station,End station,Member type,temp,atemp,hum,windspeed,distance,traffic_hour,working_day,hr,clear,mist,light_rain,heavy_rain,springer,summer,fall,winter,morning,noon,evening,night
0,0,475,7th & R St NW / Shaw Library,7th & T St NW,1,0.36,0.3788,0.66,0.0,0.192336,0,0,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0,1


In [0]:
# heatMap(df_2012_wd)

In [0]:
df_2012_predDuration = df_2012_predDuration.loc[df_2012_predDuration['distance']<30]
df_2012_predDuration.shape

(2028911, 25)

In [0]:
X = df_2012_predDuration[['hr', 'noon', 'evening','clear', 'mist',	'summer',	'fall', 'winter', "Member type", "temp", "hum", 'distance', 'traffic_hour', 'working_day']]
Y = df_2012_predDuration["Duration"]

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

Linear

In [0]:
lin_model = LinearRegression()
lin_model.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
y_train_predict = lin_model.predict(X_train)
y_test_predict = lin_model.predict(X_test)

rmse = (sqrt(mean_squared_error(Y_train, y_train_predict)))
r2 = r2_score(Y_train, y_train_predict)

print("The model performance for Traing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))


rmse = (sqrt(mean_squared_error(Y_test, y_test_predict)))
r2 = r2_score(Y_test, y_test_predict)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

The model performance for Traing set
--------------------------------------
RMSE is 1692.6079349230106
R2 score is 0.1407338726602232
The model performance for testing set
--------------------------------------
RMSE is 1724.1930817652758
R2 score is 0.13457000751593795


Polynomial


In [0]:
poly = PolynomialFeatures(degree = 3) 
X_poly = poly.fit_transform(X_train) 
  
poly.fit(X_poly, Y_train) 
lin2 = LinearRegression(n_jobs=-1)
lin2.fit(X_poly, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [0]:
y_test_predict = lin2.predict(poly.fit_transform(X_test))
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
r2 = r2_score(Y_test, y_test_predict)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

The model performance for testing set
--------------------------------------
RMSE is 1819.5441266988876
R2 score is 0.026672202464104622


SVR

In [0]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)

In [0]:
from sklearn.svm import SVR

In [0]:
regressor = SVR(kernel='linear')
regressor.fit(X_train,Y_train)

In [0]:
y_test_predict = regressor.predict(X_test )
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
r2 = r2_score(Y_test, y_test_predict)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

# Predicting Destination


In [4]:
df_2012_preddest = pd.read_csv('/content/drive/My Drive/SEM2/MP/dataset/2012_preddest_bal.csv')
df_2012_preddest.head(1)

,index,Unnamed: 0,Start station,End station,Member type,weathersit,temp,atemp,hum,windspeed,traffic_hour,working_day,season,src_label,dest_label
0,0,170197,37th & O St NW / Georgetown University,19th & E Street NW,1,1,0.4,0.4091,0.62,0.2836,1,1,1,0,0


In [0]:
X = df_2012_preddest[['weathersit','temp','hum','windspeed','traffic_hour','working_day','src_label','Member type']]
Y = df_2012_preddest['dest_label']

In [6]:
count(np.asarray(df_2012_preddest['dest_label']))

{0: 145000, 1: 145000, 2: 145000}

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [0]:
dt_gsc = GridSearchCV(
        estimator=DecisionTreeClassifier(),
        param_grid={
            'criterion': ['gini','entropy'],
            'splitter' : ['best','random'],
            'max_depth' : [15,20,25,30],
        },
        cv=5, scoring='accuracy', verbose=0, n_jobs=-1)

In [12]:
dt_grid_result = dt_gsc.fit(X_train, Y_train)
dt_best_params = dt_grid_result.best_params_
print(dt_best_params)

{'criterion': 'gini', 'max_depth': 15, 'splitter': 'random'}


In [0]:
dt_clf = DecisionTreeClassifier(criterion = "gini", splitter = 'random', max_depth = 15)
dt_clf.fit(X_train, Y_train)
dt_Y_pred = dt_clf.predict(X_test)

In [15]:
print("Confusion Matrix:\n",confusion_matrix(Y_test, dt_Y_pred))
print(classification_report(dt_Y_pred,Y_test))

Confusion Matrix:
 [[36074  5771  1573]
 [15562 27527   634]
 [10412  1619 31328]]
              precision    recall  f1-score   support

           0       0.83      0.58      0.68     62048
           1       0.63      0.79      0.70     34917
           2       0.72      0.93      0.81     33535

    accuracy                           0.73    130500
   macro avg       0.73      0.77      0.73    130500
weighted avg       0.75      0.73      0.72    130500



In [0]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
# Feature Scaling
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [0]:
#GridSearchCV
svm_gsc = GridSearchCV(
        SVC(),
        param_grid={
            'kernel': ['rbf'],
            'gamma' : ['scale','auto'],
        },
        cv=5, scoring='accuracy', verbose=0, n_jobs=-1)

In [0]:
svm_grid_result = svm_gsc.fit(X_train_sc, Y_train)
svm_best_params = svm_grid_result.best_params_
print(svm_best_params)

In [0]:
svm_clf = SVC(kernel= 'rbf')
svm_clf.fit(X_train_sc, Y_train)
svm_Y_pred = svm_clf.predict(X_test_sc)

In [0]:
print("Confusion Matrix:\n",confusion_matrix(Y_test, svm_Y_pred))
print(classification_report(Y_test, svm_Y_pred))